# Script modèle de prévision par URL CORVEN CHIFFOLEAU

In [1]:
!pip install rapidfuzz -qq

In [2]:
import pandas as pd
sub = pd.read_csv('.../sample_submission.csv')
sub.head(10)

,id,caption_title_and_reference_description
0,0,diam in ve ligula a sapien mattis a a
1,0,nisi ut eu
2,0,urna ad a duis a ligula dis ve
3,0,eros eu a vel congue ac et justo sapien
4,0,diam mi a mi nec a leo sed ornare magna mus se...
5,1,eros ad a magnis elit ad a in a
6,1,amet ut a a
7,1,pede ad
8,1,nisi eu ut est sapien felis id felis
9,1,nisl ac a eu a primis integer nih a


In [3]:
test = pd.read_csv('.../test.tsv', sep='\t')
test.head()

,id,image_url
0,0,https://upload.wikimedia.org/wikipedia/commons...
1,1,https://upload.wikimedia.org/wikipedia/commons...
2,2,https://upload.wikimedia.org/wikipedia/commons...
3,3,https://upload.wikimedia.org/wikipedia/commons...
4,4,https://upload.wikimedia.org/wikipedia/commons...


In [4]:
captions = pd.read_csv('.../test_caption_list.csv')
print(len(captions))
captions.head()

92366


,caption_title_and_reference_description
0,Albert Pike [SEP] Albert Pike
1,Anna Blount [SEP] Blount and her young daughte...
2,Río Marañón [SEP] Die Río Marañón in die Huánu...
3,Leonel Brizola [SEP] Brizola during his inaugu...
4,Buttisholz [SEP] Buttisholz


In [5]:
train0 = pd.read_csv('.../train-00000-of-00005.tsv', sep="\t", nrows=100000, engine="python")
train0.head()

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description,caption_title_and_reference_description
0,fr,https://fr.wikipedia.org/wiki/Pariser_Kanonen,https://upload.wikimedia.org/wikipedia/commons...,Pariser Kanonen,Bilan,Pariser Kanonen / Bilan,NaN,Français&#160;: Plaque apposée au n° 81 du bou...,NaN,image/jpeg,1279,1812,False,True,True,Les Pariser Kanonen sont sept pièces d’artille...,Le ou les canons restants sont démontés devant...,Pariser Kanonen [SEP]
1,tg,https://tg.wikipedia.org/wiki/Republic_P-43_La...,http://upload.wikimedia.org/wikipedia/commons/...,Republic P-43 Lancer,NaN,Republic P-43 Lancer,NaN,A Republic P-43 Lancer in flight over Esler Fi...,NaN,image/jpeg,306,591,True,False,False,Republic P-43 Lancer — як ҳавогарди сохтаи Rep...,Republic P-43 Lancer (англ. Republic P-43 Lanc...,Republic P-43 Lancer [SEP]
2,en,"https://en.wikipedia.org/wiki/Deer_Park,_Wisco...",https://upload.wikimedia.org/wikipedia/commons...,"Deer Park, Wisconsin",NaN,"Deer Park, Wisconsin",Downtown Deer Park,"English: Downtown Deer Park, Wisconsin on WIS46.",Downtown Deer Park,image/jpeg,1824,2736,True,True,True,"Deer Park is a village in St. Croix County, Wi...","Deer Park is a village in St. Croix County, Wi...","Deer Park, Wisconsin [SEP] Downtown Deer Park"
3,pt,https://pt.wikipedia.org/wiki/Vaux-l%C3%A8s-Mo...,https://upload.wikimedia.org/wikipedia/commons...,Vaux-lès-Mouzon,NaN,Vaux-lès-Mouzon,NaN,Français&#160;: Vaux-lès-Mouzon&#160;: le village,NaN,image/jpeg,1704,2272,True,False,False,Vaux-lès-Mouzon é uma comuna francesa na regiã...,Vaux-lès-Mouzon é uma comuna francesa na regiã...,Vaux-lès-Mouzon [SEP]
4,ne,https://ne.wikipedia.org/wiki/%E0%A4%95%E0%A5%...,https://upload.wikimedia.org/wikipedia/commons...,क्रिसमस टापु,NaN,क्रिसमस टापु,NaN,This is a locator map for Christmas Island I m...,NaN,image/png,165,248,True,False,False,क्रिसमस टापु हिन्द महासागरमा अवस्थित अष्ट्रेलि...,क्रिसमस टापु हिन्द महासागरमा अवस्थित अष्ट्रेलि...,क्रिसमस टापु [SEP]


In [7]:
from urllib.parse import unquote

t = train0.image_url.loc[3]

def convert(t):
    t = t.rsplit('/',1)[1]
    t = unquote(t)
    t = t.replace('_', ' ')
    t = t + ' [SEP]'
    return(t)

In [8]:
test['prediction'] = test['image_url'].apply(convert)

In [9]:
test.head()

,id,image_url,prediction
0,0,https://upload.wikimedia.org/wikipedia/commons...,Scots Gaelic speakers in the 2011 census.png [...
1,1,https://upload.wikimedia.org/wikipedia/commons...,Thermopylae ancient coastline large.jpg [SEP]
2,2,https://upload.wikimedia.org/wikipedia/commons...,Map of New York highlighting Sullivan County.s...
3,3,https://upload.wikimedia.org/wikipedia/commons...,Kykkos Watermill.jpg [SEP]
4,4,https://upload.wikimedia.org/wikipedia/commons...,Kansai International Airport03s3s4410.jpg [SEP]


In [10]:
CAPTIONS = captions.caption_title_and_reference_description.values.tolist()
len(CAPTIONS)


92366

In [11]:
from rapidfuzz import process, fuzz

In [12]:
%%time

for i in range(5):
    s = test.prediction.loc[i]
    print(f'image_url: {s}')
    res = process.extract(s, CAPTIONS, scorer=fuzz.ratio, processor=None, limit=5)
    print(f'closest captions:')
    for c in res:
        print(c[0])
    print('*'*60)
    print()  

image_url: Scots Gaelic speakers in the 2011 census.png [SEP]
closest captions:
São Vicente do Penso [SEP] Escudo
Jocs Panamericans de 2011 [SEP] Guadalajara.
Bois de pins et chênes de Madrean [SEP] carte
Standard time in the United States [SEP] 1913
Hebrides [SEP] Geographic distribution of Gaelic speakers in Scotland (2011)
************************************************************

image_url: Thermopylae ancient coastline large.jpg [SEP]
closest captions:
Oberlin College [SEP] Logo
Departamento de Flores [SEP] Escudo
Sekolah Menengah Kebangsaan Selandar [SEP] CNY 4
Academia de Ciencias de Cuba [SEP] Sede.
Geothermal areas in New Zealand [SEP] Geyser Flat
************************************************************

image_url: Map of New York highlighting Sullivan County.svg [SEP]
closest captions:
Breaking Clean [SEP] Map of Montana highlighting Phillips County
New York City Subway [SEP] Ein Token
Deogyang-gu [SEP] Map of Gyeonggi highlighting Deogyang-gu.
Rome, New York [SEP] Loc

In [13]:
def find_closest_match(s):
    res = process.extract(s, CAPTIONS, scorer=fuzz.ratio, processor=None, limit=5)
    res = [x[0] for x in res]
    return res

In [14]:
from tqdm.auto import tqdm
tqdm.pandas()

In [15]:
test['caption_title_and_reference_description'] = test['prediction'].progress_apply(find_closest_match)

  0%|          | 0/92366 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
sub = test[['id', 'caption_title_and_reference_description']]
sub = sub.explode('caption_title_and_reference_description')
sub.head()

KeyError: "['caption_title_and_reference_description'] not in index"

In [41]:
sub.to_csv('submission2222.csv', index=False)